# Time series analysis of electricity usage

So Cal Edison provides hour-by-hour power consumption for its customers. Let's explore using Python, Pandas and Plotly.

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import re

In [80]:
# Load usage from excel
xlsxpath = "E:/UserLo/source/repos/learning/SCEdison/"
xlsx = "SCE_Usage_8000272817_04-01-21_to_04-01-22.xlsx"
energy_usage = "SCEUsageClean"

energy_usage_df = pd.read_excel(xlsxpath+xlsx,sheet_name=energy_usage)
energy_usage_df.rename(columns={'Energy   Consumption  time period':'energy_consumption_time_period','Usage in kilowatt-hours':'usage_in_kilowatt_hours'},inplace=True)
energy_usage_df.head()

,energy_consumption_time_period,usage_in_kilowatt_hours
0,2021-04-01 00:00:00 to 2021-04-01 01:00:00,0.46
1,2021-04-01 01:00:00 to 2021-04-01 02:00:00,0.33
2,2021-04-01 02:00:00 to 2021-04-01 03:00:00,0.24
3,2021-04-01 03:00:00 to 2021-04-01 04:00:00,0.26
4,2021-04-01 04:00:00 to 2021-04-01 05:00:00,0.25


In [7]:
energy_usage_df.dtypes

Energy   Consumption  time period     object
Usage in kilowatt-hours              float64
dtype: object

Note that the string format for energy_consumption_time_period contains a \xa0 character (unicode non-breaking space) before the word 'to', however, the regex '\s' handles it correctly has whitespace.

In [45]:
energy_usage_df.iloc[0,0]

'2021-04-01 00:00:00\xa0to 2021-04-01 01:00:00'

### Split the energy_consumption_time_period

Split energy_consumption_time_period into start_dttm and end_dttm and concat

Try it first with just the head.

In [4]:
# Try one
# Split text column "Energy Consumption time period" into start_dt and end_dt
splitregex = re.compile('\sto\s')
start, end = splitregex.split(energy_usage_df.iloc[0,0])
print(f'{start=} {end=}')

start='2021-04-01 00:00:00' end='2021-04-01 01:00:00'


In [36]:
ehead_df = energy_usage_df.head()
ehead_df

,energy_consumption_time_period,usage_in_kilowatt_hours
0,2021-04-01 00:00:00 to 2021-04-01 01:00:00,0.46
1,2021-04-01 01:00:00 to 2021-04-01 02:00:00,0.33
2,2021-04-01 02:00:00 to 2021-04-01 03:00:00,0.24
3,2021-04-01 03:00:00 to 2021-04-01 04:00:00,0.26
4,2021-04-01 04:00:00 to 2021-04-01 05:00:00,0.25


In [50]:
ehead_df['energy_consumption_time_period'].str.split(pat=splitregex,expand=True).rename(columns={0:'start_dttm',1:'end_dttm'})

,start_dttm,end_dttm
0,2021-04-01 00:00:00,2021-04-01 01:00:00
1,2021-04-01 01:00:00,2021-04-01 02:00:00
2,2021-04-01 02:00:00,2021-04-01 03:00:00
3,2021-04-01 03:00:00,2021-04-01 04:00:00
4,2021-04-01 04:00:00,2021-04-01 05:00:00


In [ ]:
ehead_df = pd.concat([ehead_df,\
    ehead_df['energy_consumption_time_period'].str.split(pat=splitregex,expand=True).rename(columns={0:'start_dttm',1:'end_dttm'})], axis=1)
ehead_df

In [55]:
ehead_df.dtypes

energy_consumption_time_period     object
usage_in_kilowatt_hours           float64
start_dttm                         object
end_dttm                           object
dtype: object

In [69]:
# Change the data types of the dttm columns
ehead_df = ehead_df.astype({'start_dttm':'datetime64','end_dttm':'datetime64'})
ehead_df.dtypes

energy_consumption_time_period            object
usage_in_kilowatt_hours                  float64
start_dttm                        datetime64[ns]
end_dttm                          datetime64[ns]
dtype: object

In [114]:
# Day of week starts on Monday = 0
ehead_df.loc[0,'start_dttm'].day_of_week

3

### Perform split on entire DataFrame

It works on a sample, let's do the entire energy_usage_df.

In [82]:
energy_usage_df = pd.concat([energy_usage_df,\
    energy_usage_df['energy_consumption_time_period'].str.split(pat=splitregex,expand=True).rename(columns={0:'start_dttm',1:'end_dttm'})], axis=1)

energy_usage_df.head()

,energy_consumption_time_period,usage_in_kilowatt_hours,start_dttm,end_dttm
0,2021-04-01 00:00:00 to 2021-04-01 01:00:00,0.46,2021-04-01 00:00:00,2021-04-01 01:00:00
1,2021-04-01 01:00:00 to 2021-04-01 02:00:00,0.33,2021-04-01 01:00:00,2021-04-01 02:00:00
2,2021-04-01 02:00:00 to 2021-04-01 03:00:00,0.24,2021-04-01 02:00:00,2021-04-01 03:00:00
3,2021-04-01 03:00:00 to 2021-04-01 04:00:00,0.26,2021-04-01 03:00:00,2021-04-01 04:00:00
4,2021-04-01 04:00:00 to 2021-04-01 05:00:00,0.25,2021-04-01 04:00:00,2021-04-01 05:00:00


In [74]:
energy_usage_df.dtypes

energy_consumption_time_period     object
usage_in_kilowatt_hours           float64
start_dttm                         object
end_dttm                           object
dtype: object

In [84]:
# Change the data types of the dttm columns
energy_usage_df = energy_usage_df.astype({'start_dttm':'datetime64','end_dttm':'datetime64'})
energy_usage_df.dtypes

energy_consumption_time_period            object
usage_in_kilowatt_hours                  float64
start_dttm                        datetime64[ns]
end_dttm                          datetime64[ns]
dtype: object

### Let's see a fine-grained look of the usage over the 12 months

Can clearly see blocks of time with low power usage while I was away on vacation.

In [105]:
# On the finest grain level, show the hour-by-hour usage over the 12 months
fig=px.line(energy_usage_df,x='start_dttm',y='usage_in_kilowatt_hours')
fig.update_traces(line={'color':'darksalmon'})
fig.show()

In [85]:
energy_usage_df['start_dt'] = energy_usage_df['start_dttm'].apply(lambda date: date.floor(freq='D'))

In [86]:
energy_usage_df['start_day'] = energy_usage_df['start_dttm'].apply(lambda date: date.day_of_week)
energy_usage_df.head()

,energy_consumption_time_period,usage_in_kilowatt_hours,start_dttm,end_dttm,start_dt,start_day
0,2021-04-01 00:00:00 to 2021-04-01 01:00:00,0.46,2021-04-01 00:00:00,2021-04-01 01:00:00,2021-04-01,3
1,2021-04-01 01:00:00 to 2021-04-01 02:00:00,0.33,2021-04-01 01:00:00,2021-04-01 02:00:00,2021-04-01,3
2,2021-04-01 02:00:00 to 2021-04-01 03:00:00,0.24,2021-04-01 02:00:00,2021-04-01 03:00:00,2021-04-01,3
3,2021-04-01 03:00:00 to 2021-04-01 04:00:00,0.26,2021-04-01 03:00:00,2021-04-01 04:00:00,2021-04-01,3
4,2021-04-01 04:00:00 to 2021-04-01 05:00:00,0.25,2021-04-01 04:00:00,2021-04-01 05:00:00,2021-04-01,3


Sum the usage by date and plot it

In [70]:
energy_usage_daily_df = energy_usage_df.groupby("start_dt").sum()
energy_usage_daily_df.head()

,usage_in_kilowatt_hours
start_dt,
2021-04-01,10.45
2021-04-02,10.43
2021-04-03,10.15
2021-04-04,13.07
2021-04-05,12.20


Rather than using the lambda function to create a new start_dt column, the groupby could have used the dt accessor.

Note: df.groupby('A') is just syntactic sugar for df.groupby(df['A').

In [121]:
energy_usage_daily_dfx = energy_usage_df.groupby(energy_usage_df["start_dttm"].dt.date).sum()
energy_usage_daily_dfx.head(5)

,usage_in_kilowatt_hours,start_day
start_dttm,,
2021-04-01,10.45,72
2021-04-02,10.43,96
2021-04-03,10.15,120
2021-04-04,13.07,144
2021-04-05,12.20,0


In [72]:
fig=px.line(energy_usage_daily_df,y='usage_in_kilowatt_hours')
fig.update_traces(line={'color':'darksalmon'})
fig.show()

Compute day of week usage and normalized day of week usage and plot it

In [87]:
energy_usage_day_of_week_df = energy_usage_df.groupby("start_day").sum()
energy_usage_day_of_week_df.head()

,usage_in_kilowatt_hours
start_day,
0,520.49
1,533.92
2,520.35
3,531.22
4,529.23


Same thing using dt accessor

See:
https://pandas.pydata.org/docs/getting_started/intro_tutorials/09_timeseries.html?highlight=frequency

In [103]:
energy_usage_day_of_week_dfx = energy_usage_df.groupby(energy_usage_df["start_dttm"].dt.day_of_week).sum()
energy_usage_day_of_week_dfx.head(20)

,usage_in_kilowatt_hours,start_day
start_dttm,,
0,520.49,0
1,533.92,1248
2,520.35,2496
3,531.22,3816
4,529.23,5088
5,519.08,6240
6,513.94,7488


In [88]:
mean_day_usage = energy_usage_day_of_week_df['usage_in_kilowatt_hours'].mean()
energy_usage_day_of_week_df['usage_diff']= energy_usage_day_of_week_df['usage_in_kilowatt_hours']-mean_day_usage
energy_usage_day_of_week_df.head()

,usage_in_kilowatt_hours,usage_diff
start_day,,
0,520.49,-3.542857
1,533.92,9.887143
2,520.35,-3.682857
3,531.22,7.187143
4,529.23,5.197143


In [90]:
energy_usage_day_of_week_df.index=energy_usage_day_of_week_df.index.map({0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'})

In [91]:
energy_usage_day_of_week_df

,usage_in_kilowatt_hours,usage_diff
start_day,,
Monday,520.49,-3.542857
Tuesday,533.92,9.887143
Wednesday,520.35,-3.682857
Thursday,531.22,7.187143
Friday,529.23,5.197143
Saturday,519.08,-4.952857
Sunday,513.94,-10.092857


In [93]:
fig=px.bar(energy_usage_day_of_week_df,y='usage_diff')
#fig.update_traces(line={'color':'darksalmon'})
fig.show()

In [100]:
energy_usage_daily_df[(energy_usage_daily_df.index>=pd.Timestamp('2022-03')) & (energy_usage_daily_df.index<pd.Timestamp('2022-04'))]

,usage_in_kilowatt_hours
start_dt,
2022-03-01,9.97
2022-03-02,8.93
2022-03-03,9.43
2022-03-04,11.25
2022-03-05,7.65
2022-03-06,5.35
2022-03-07,5.34
2022-03-08,5.54
2022-03-09,5.51


Make a new index combining year and month

In [118]:
yr_month_idx = energy_usage_daily_df.index.year.astype('str') + energy_usage_daily_df.index.month_name()
yr_month_idx

Index(['2021April', '2021April', '2021April', '2021April', '2021April',
       '2021April', '2021April', '2021April', '2021April', '2021April',
       ...
       '2022March', '2022March', '2022March', '2022March', '2022March',
       '2022March', '2022March', '2022March', '2022March', '2022April'],
      dtype='object', name='start_dt', length=366)

In [122]:
energy_usage_daily_df.index = yr_month_idx
energy_usage_daily_df.head()

,usage_in_kilowatt_hours
start_dt,
2021April,10.45
2021April,10.43
2021April,10.15
2021April,13.07
2021April,12.20


Find the unique values which will then be used as columns in new DF that has day number as index and YrMonth as columns

In [139]:
unique_yr_month = yr_month_idx.unique()
unique_yr_month_lst = unique_yr_month.to_list()


Create an empty dataframe,energy_usage_daily_by_yr_df, and populate it with the daily usage for each yr month

In [129]:
energy_usage_daily_by_yr_df = pd.DataFrame(index=range(0,31,1))

In [140]:
for yr_mo in unique_yr_month_lst:
    energy_usage_daily_by_yr_df = pd.concat([energy_usage_daily_by_yr_df,energy_usage_daily_df.loc[yr_mo].reset_index(drop=True)],axis=1).\
        rename(columns={'usage_in_kilowatt_hours':yr_mo})

In [141]:
energy_usage_daily_by_yr_df.head()

,2021April,2021May,2021June,2021July,2021August,2021September,2021October,2021November,2021December,2022January,2022February,2022March,2022April
0,10.45,10.26,10.34,9.24,10.26,6.43,10.34,12.07,13.53,9.86,11.82,9.97,9.95
1,10.43,10.85,10.34,9.68,9.07,8.35,10.67,10.50,15.24,9.63,10.34,8.93,NaN
2,10.15,10.39,10.16,9.73,10.47,9.38,10.93,8.78,11.17,11.22,9.14,9.43,NaN
3,13.07,10.19,10.43,8.70,9.13,10.77,11.02,8.72,14.73,11.98,10.54,11.25,NaN
4,12.20,10.91,9.39,9.61,10.19,11.63,10.14,9.91,11.02,10.06,10.17,7.65,NaN


In [142]:
energy_usage_daily_by_yr_df.tail()

,2021April,2021May,2021June,2021July,2021August,2021September,2021October,2021November,2021December,2022January,2022February,2022March,2022April
26,11.48,9.79,9.39,8.49,6.31,9.27,9.43,11.22,12.15,11.57,9.73,10.42,NaN
27,11.53,10.33,10.30,9.13,6.30,8.96,9.51,12.82,9.25,10.03,9.79,11.45,NaN
28,10.38,10.09,9.16,9.52,6.51,9.76,9.75,9.84,10.17,10.77,NaN,11.54,NaN
29,9.61,10.70,10.25,10.54,6.26,9.89,9.15,10.34,9.46,9.94,NaN,9.73,NaN
30,NaN,9.55,NaN,10.08,6.30,NaN,10.14,NaN,9.50,11.42,NaN,10.50,NaN


In [145]:
# April2022 is almost empty so drop it
energy_usage_daily_by_yr_df.drop("2022April",axis=1,inplace=True)
energy_usage_daily_by_yr_df.head()

,2021April,2021May,2021June,2021July,2021August,2021September,2021October,2021November,2021December,2022January,2022February,2022March
0,10.45,10.26,10.34,9.24,10.26,6.43,10.34,12.07,13.53,9.86,11.82,9.97
1,10.43,10.85,10.34,9.68,9.07,8.35,10.67,10.50,15.24,9.63,10.34,8.93
2,10.15,10.39,10.16,9.73,10.47,9.38,10.93,8.78,11.17,11.22,9.14,9.43
3,13.07,10.19,10.43,8.70,9.13,10.77,11.02,8.72,14.73,11.98,10.54,11.25
4,12.20,10.91,9.39,9.61,10.19,11.63,10.14,9.91,11.02,10.06,10.17,7.65


In [150]:
energy_usage_daily_by_yr_corr_df = energy_usage_daily_by_yr_df.corr()

In [151]:
energy_usage_daily_by_yr_corr_df

,2021April,2021May,2021June,2021July,2021August,2021September,2021October,2021November,2021December,2022January,2022February,2022March
2021April,1.000000,0.066363,0.315330,-0.282603,0.344498,0.359151,0.094096,-0.268932,0.041724,0.039274,-0.108407,-0.215544
2021May,0.066363,1.000000,-0.182336,0.007029,0.268117,0.108112,0.125422,-0.420754,0.308672,-0.296321,-0.071376,-0.209343
2021June,0.315330,-0.182336,1.000000,0.138705,0.261798,-0.135785,0.053265,0.132070,-0.240375,-0.031214,-0.130956,-0.280241
2021July,-0.282603,0.007029,0.138705,1.000000,0.122319,0.201703,0.153141,-0.051173,-0.268910,-0.095891,-0.257968,-0.221055
2021August,0.344498,0.268117,0.261798,0.122319,1.000000,0.225812,0.264025,-0.387684,-0.038160,-0.217930,-0.436067,-0.640685
2021September,0.359151,0.108112,-0.135785,0.201703,0.225812,1.000000,0.248102,-0.301951,-0.169006,-0.062995,-0.371026,-0.411744
2021October,0.094096,0.125422,0.053265,0.153141,0.264025,0.248102,1.000000,0.034061,-0.222123,-0.093852,0.159529,-0.418422
2021November,-0.268932,-0.420754,0.132070,-0.051173,-0.387684,-0.301951,0.034061,1.000000,-0.204862,-0.008417,0.149814,0.105483
2021December,0.041724,0.308672,-0.240375,-0.268910,-0.038160,-0.169006,-0.222123,-0.204862,1.000000,0.097943,0.378976,0.531332
2022January,0.039274,-0.296321,-0.031214,-0.095891,-0.217930,-0.062995,-0.093852,-0.008417,0.097943,1.000000,0.233010,0.313663


The daily pattern of monthly usage is not correlated with other months

# Original Data
grouped = ts.groupby(lambda x: x.year)

grouped.mean()
Out[108]: 
2000    0.442441
2001    0.526246
2002    0.459365

### Predict final 9 days of energy usage

One of the objectives of this study is to use time series analysis to predict the energy usage in the final 9 days of the billing cycle.

Augment hour-by-hour to determine which billing cycle it belongs to.

Augment data with day of week to help determine if there a usage pattern based on day of week.

Identify periods of anomalous usage, such as during vacations, that will impact weekly or monthly averages.

Look for long-term trends using weekly, monthly and quartely averages.


In [120]:
# Day of week starts on Monday = 0
energy_usage_df.loc[0,'start_dttm'].day_of_week

3

Read the billing Excel to build the bill intervals

In [160]:
# Load the billing and projections Excel sheets
xlsxpath = "E:/UserLo/source/repos/learning/SCEdison/"
xlsx = "SoCalEdison Bills.xlsx"
billing_sheet = "Bills"

bills_df = pd.read_excel(xlsxpath+xlsx,sheet_name=billing_sheet)

In [161]:
bills_interval_df = bills_df[['date','amt']]

In [162]:
bills_interval_s = pd.Series(data=[np.nan]*bills_interval_df.index.stop,dtype='object',name='bill_interval')
for i,r in bills_interval_df.iterrows():
    bills_interval_s.iloc[i] = pd.Interval(bills_interval_df.iloc[i+1,0],r['date'])

    # Can't make interval on last row because there is no date previous to that one
    if i == (bills_interval_df.index.stop)-2:
        break

# Drop the last row with NaN interval
bills_interval_df = pd.concat([bills_interval_df,bills_interval_s],axis=1).dropna()
bills_interval_df


,date,amt,bill_interval
0,2022-04-21,30.17,"(2022-03-22, 2022-04-21]"
1,2022-03-22,81.48,"(2022-02-18, 2022-03-22]"
2,2022-02-18,72.73,"(2022-01-21, 2022-02-18]"
3,2022-01-21,89.43,"(2021-12-20, 2022-01-21]"
4,2021-12-20,90.26,"(2021-11-19, 2021-12-20]"
5,2021-11-19,69.37,"(2021-10-20, 2021-11-19]"
6,2021-10-20,44.22,"(2021-09-20, 2021-10-20]"
7,2021-09-20,69.65,"(2021-08-19, 2021-09-20]"
8,2021-08-19,67.26,"(2021-07-21, 2021-08-19]"
9,2021-07-21,75.01,"(2021-06-21, 2021-07-21]"


Drop rows 0,12,13,14 since I don't have usage data for those billing intervals

In [163]:
bills_interval_with_usage_df = bills_interval_df.drop([0,12,13,14], axis=0)
bills_interval_with_usage_df.reset_index(drop=True,inplace=True)
bills_interval_with_usage_df


,date,amt,bill_interval
0,2022-03-22,81.48,"(2022-02-18, 2022-03-22]"
1,2022-02-18,72.73,"(2022-01-21, 2022-02-18]"
2,2022-01-21,89.43,"(2021-12-20, 2022-01-21]"
3,2021-12-20,90.26,"(2021-11-19, 2021-12-20]"
4,2021-11-19,69.37,"(2021-10-20, 2021-11-19]"
5,2021-10-20,44.22,"(2021-09-20, 2021-10-20]"
6,2021-09-20,69.65,"(2021-08-19, 2021-09-20]"
7,2021-08-19,67.26,"(2021-07-21, 2021-08-19]"
8,2021-07-21,75.01,"(2021-06-21, 2021-07-21]"
9,2021-06-21,74.58,"(2021-05-20, 2021-06-21]"


Add a Multi-index identifying which billing interval the usage belongs to and the billing cycle day. Actual day of the month is not important.

In [166]:
(bills_interval_with_usage_df.iloc[0,0]).month

3

Create a DataFrame with days of billing cycle as index and columns